<img src='logo/dsl-logo.png' width="500" align="center" />

# Our First HR Competition

## Load Data

### Initializations

Es werden die benötigten Bibliotheken eingebunden und für Variationsmöglichkeiten bei der Ausgabe wird class color definiert. 

In [1]:
# Bibliotheken einbinden
import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Definition einer Klasse für Text Styles
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Import Data from CSV
Die Trainings- und Testdaten werden nacheinander geladen, mit dem Delimiter **Komma** und dem **Punkt** als Dezimalstellle. Die Spaltenköpfe sowie das Format der Daten werden überprüft und angepasst. 

#### Einlesen der Trainingsdaten
Bei den Trainingsdaten wird die Spalte ID nicht benötigt und daher entfernt.


In [8]:
# Trainingsdaten einlesen
df = pd.read_csv('data/hr_train.csv', delimiter=',', decimal='.')
df.drop('id', axis=1, inplace=True)
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.65,0.96,5,226,2,1,0,0,marketing,medium
1,0.88,0.80,3,166,2,0,0,0,IT,low
2,0.69,0.98,3,214,2,0,0,0,sales,low
3,0.41,0.47,2,154,3,0,1,0,sales,low
4,0.87,0.76,5,254,2,1,0,0,hr,low


In [9]:
# Spaltenname angepasst
df = df.rename(columns={
    'satisfaction_level': 'satisfactionLevel',
    'last_evaluation': 'yearsSinceEvaluation',
    'number_project': 'numberOfProjects',
    'average_montly_hours': 'averageMonthlyHours',
    'time_spend_company': 'yearsAtCompany',
    'Work_accident': 'workAccident',
    'left': 'hasLeftCompany',
    'promotion_last_5years': 'gotPromotion',
    'department': 'department',
    'salary': 'salary'
})

In [10]:
# Format des Trainingsdaten Dataframes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
satisfactionLevel       10000 non-null float64
yearsSinceEvaluation    10000 non-null float64
numberOfProjects        10000 non-null int64
averageMonthlyHours     10000 non-null int64
yearsAtCompany          10000 non-null int64
workAccident            10000 non-null int64
hasLeftCompany          10000 non-null int64
gotPromotion            10000 non-null int64
department              10000 non-null object
salary                  10000 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 781.3+ KB


#### Einlesen der Testdaten

In [12]:
# Testdaten einlesen
df_test = pd.read_csv('data/hr_test.csv', delimiter=',', decimal='.')
df_test.head()

,id,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary
0,10000,0.81,0.96,4,219,2,0,0,technical,low
1,10001,0.86,0.84,4,246,6,0,0,IT,low
2,10002,0.90,0.66,4,242,3,0,0,support,high
3,10003,0.37,0.54,2,131,3,1,0,hr,medium
4,10004,0.52,0.96,3,271,3,1,0,technical,medium


In [13]:
# Spaltenname angepasst
df_test = df_test.rename(columns={
    'satisfaction_level': 'satisfactionLevel',
    'last_evaluation': 'yearsSinceEvaluation',
    'number_project': 'numberOfProjects',
    'average_montly_hours': 'averageMonthlyHours',
    'time_spend_company': 'yearsAtCompany',
    'Work_accident': 'workAccident',
    'promotion_last_5years': 'gotPromotion',
    'department': 'department',
    'salary': 'salary'
})

In [14]:
# Format des Testdaten Dataframes
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 10 columns):
id                      4999 non-null int64
satisfactionLevel       4999 non-null float64
yearsSinceEvaluation    4999 non-null float64
numberOfProjects        4999 non-null int64
averageMonthlyHours     4999 non-null int64
yearsAtCompany          4999 non-null int64
workAccident            4999 non-null int64
gotPromotion            4999 non-null int64
department              4999 non-null object
salary                  4999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 390.6+ KB


### Check auf passende Datentypen

Detailliertere Analyse der Daten, dabei soll festgestellt werden ob das Format der Daten angepasst werden muss, um z.B. ein speicheroptimiertes Format auszuwählen. 

In [15]:
# Anzahl Unique Values pro Column
print(color.UNDERLINE + color.BOLD + 'Number of Unique Values per Column:' + color.END)
for col in df.columns:
    col_unique_value_count = df[col].unique().size
    print('\t' + col + color.BOLD,':', col_unique_value_count, color.END)

Number of Unique Values per Column:
	satisfactionLevel : 92 
	yearsSinceEvaluation : 65 
	numberOfProjects : 6 
	averageMonthlyHours : 215 
	yearsAtCompany : 8 
	workAccident : 2 
	hasLeftCompany : 2 
	gotPromotion : 2 
	department : 10 
	salary : 3 


In [16]:
df.columns

Index(['satisfactionLevel', 'yearsSinceEvaluation', 'numberOfProjects',
       'averageMonthlyHours', 'yearsAtCompany', 'workAccident',
       'hasLeftCompany', 'gotPromotion', 'department', 'salary'],
      dtype='object')

In [17]:
# Ausgabe der einzelnen Ausprägungen und deren Anzahl pro Spalte mit höchstens 11 Unique Values
for col in df.columns:
    if df[col].unique().size <= 10:
        print()
        print(color.BOLD + ' attr : count for ' + col + color.END)
        uniques, counts = np.unique(df[col], return_counts=True)
        zipped = zip(uniques, counts)
        dictionary = dict(zipped)
        pd.DataFrame.from_dict(data=dictionary, orient='index')
        dictionary
        for unique, count in dictionary.items():
            print('\t', unique, ':', count)


 attr : count for numberOfProjects
	 2 : 1606
	 3 : 2693
	 4 : 2920
	 5 : 1825
	 6 : 781
	 7 : 175

 attr : count for yearsAtCompany
	 2 : 2212
	 3 : 4268
	 4 : 1692
	 5 : 1002
	 6 : 476
	 7 : 112
	 8 : 97
	 10 : 141

 attr : count for workAccident
	 0 : 8563
	 1 : 1437

 attr : count for hasLeftCompany
	 0 : 7615
	 1 : 2385

 attr : count for gotPromotion
	 0 : 9804
	 1 : 196

 attr : count for department
	 IT : 810
	 RandD : 509
	 accounting : 502
	 hr : 505
	 management : 375
	 marketing : 586
	 product_mng : 601
	 sales : 2763
	 support : 1495
	 technical : 1854

 attr : count for salary
	 high : 814
	 low : 4912
	 medium : 4274


### Anpassen der Datentypen

Für die Spalten workAccident, hasLeftCompany, gotPromotion, department und salary ist es sinnvoll den Typ ‘Category‘ zu verwenden, dies ermöglicht eine Optimierung bei der Speicherung. Im Anschluss wird die Umwandlung noch überprüft.

In [18]:
# Datentyp für bestimmte Spalten in Category umwandeln
for col in ['workAccident', 'hasLeftCompany', 'gotPromotion', 'department', 'salary']:
    print('transforming', col)
    df[col] = df[col].astype('category')
    if col != 'hasLeftCompany':
        df_test[col] = df_test[col].astype('category')

transforming workAccident
transforming hasLeftCompany
transforming gotPromotion
transforming department
transforming salary


In [19]:
# Ergebnis überprüfen
print(color.BOLD + 'Info about HR Training Dataframe' + color.END)
print(df.info())
print()
print(color.BOLD + 'Info about HR Test Dataframe' + color.END)
print(df_test.info())

Info about HR Training Dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
satisfactionLevel       10000 non-null float64
yearsSinceEvaluation    10000 non-null float64
numberOfProjects        10000 non-null int64
averageMonthlyHours     10000 non-null int64
yearsAtCompany          10000 non-null int64
workAccident            10000 non-null category
hasLeftCompany          10000 non-null category
gotPromotion            10000 non-null category
department              10000 non-null category
salary                  10000 non-null category
dtypes: category(5), float64(2), int64(3)
memory usage: 440.3 KB
None

Info about HR Test Dataframe
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 10 columns):
id                      4999 non-null int64
satisfactionLevel       4999 non-null float64
yearsSinceEvaluation    4999 non-null float64
numberOfProjects        4999 non-null int64
ave

### Export Dataframe for Next Steps
Die Aufbereiteten Daten werden nun exportiert, dabei wird das Dateiformat **.pkl** gewählt, um gleich mit dem formatierten Dataframe weiter zu arbeiten.

In [20]:
df.to_pickle('exchange/hr_01_loaded_train.pkl')
df_test.to_pickle('exchange/hr_01_loaded_test.pkl')